In [1]:
!pip install keras keras_nlp tensorflow tensorflow-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 24.7 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Suc

In [2]:
import keras_nlp
import keras
import tensorflow as tf
import time

keras.mixed_precision.set_global_policy("mixed_float16")

In [3]:
preprocessor = keras_nlp.models.GPT2CausalLMPreprocessor.from_preset(
    "gpt2_base_en",
    sequence_length=128,)

100%|██████████| 448/448 [00:00<00:00, 571kB/s]
100%|██████████| 0.99M/0.99M [00:00<00:00, 5.55MB/s]
100%|██████████| 446k/446k [00:00<00:00, 3.12MB/s]


In [4]:
gpt2_lm = keras_nlp.models.GPT2CausalLM.from_preset(
    "gpt2_base_en",
    preprocessor = preprocessor
)

100%|██████████| 484/484 [00:00<00:00, 576kB/s]
100%|██████████| 475M/475M [00:04<00:00, 104MB/s]


In [5]:
start = time.time()

output = gpt2_lm.generate("My trip to USA was ", max_length=100)
print("\nOutput:",)
print(output)

end = time.time()
print(f"Time elapsed: {end - start:.2f}s")


Output:
My trip to USA was  a little bit longer than I expected. I went to the airport on my way out of the US and was greeted by a man who was wearing a T-shirt that read: "I'm from America and I've been here for a while. I've been here a long time and I've been here for a long time, but I don't know if it's time for my return." I'm not sure why he was so excited about it,
Time elapsed: 35.80s


In [6]:
output = gpt2_lm.generate("Climbing Everest was like ", max_length=50)
print(output)

Climbing Everest was like  a dream. I was so excited.
The first time I climbed Everest, I had a lot of anxiety. The first thing I did was go down the steep, dark, and dangerous mountain and I


In [7]:
import tensorflow_datasets as tfds

In [8]:
dataset = tfds.load("race", split="train")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/18728 [00:00<?, ? examples/s]

...

Generating dev examples...:   0%|          | 0/1021 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/race/high/2.0.0.incompleteQYL9XA/race-dev.tfrecord*...:   0%|          | 0…

Generating test examples...:   0%|          | 0/1045 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/race/high/2.0.0.incompleteQYL9XA/race-test.tfrecord*...:   0%|          | …

Dataset race downloaded and prepared to /root/tensorflow_datasets/race/high/2.0.0. Subsequent calls will reuse this data.


In [10]:
ds_list = []

def process_element(record):
  return record["article"].numpy().decode("utf-8")
for ele in dataset:
  ds_list.append(process_element(ele))


print(ds_list)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [11]:
num_epochs=1

learning_rate = keras.optimizers.schedules.PolynomialDecay(
    5e-5,
    decay_steps=dataset.cardinality() * num_epochs,
    end_learning_rate=0.0
)

In [12]:
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [13]:
gpt2_lm.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=loss,
    weighted_metrics=["accuracy"],
)


In [ ]:
gpt2_lm.fit(ds_list, epochs=num_epochs)

  8/586 ━━━━━━━━━━━━━━━━━━━━ 5:03:24 31s/step - accuracy: 0.3364 - loss: 3.6427